In [1]:
import sys
sys.path.append('../mwtransits')
import mcmc
import pymc3 as pm
import theano.tensor as tt
from exoplanet.gp import terms, GP
import numpy as np
import utils
import exoplanet as xo
import matplotlib.pyplot as pl
import corner
%matplotlib inline

red = '#FE4365'
blue = '#00A9FF'
yellow = '#ECA25C'
green = '#3F9778'
darkblue = '#005D7F'

pl.rc('xtick', labelsize=20)
pl.rc('ytick', labelsize=20)
pl.rc('axes', labelsize=25)
pl.rc('axes', titlesize=30)
pl.rc('legend', fontsize=20)
pl.rc('lines', linewidth=4)

t = np.linspace(-5, 5, 3000)
tparams = [0.0, 0.1, 0.5, 0.05]  # t0, r, d, tin

alpha = -13 # total variance 
x = np.linspace(-3, 3, 20)
logr = np.log(10 ** x)
logsig = 0.5 * (alpha - np.log(1 + np.exp(2*logr)))
logs0 = logr + logsig
logq = np.log(1/np.sqrt(2))
diag = np.exp(2*logsig)
a = [1, 2]

w0T = 1.0
logw0 = np.log(w0T) - np.log(tparams[2])
logs0 = 2 * logs0  - logw0

unc1d = np.zeros((4, len(logr)))
unc2d = np.zeros((4, len(logr)))

In [2]:
y2d, y1d = mcmc.make_data(t, logs0[-1] + 2*np.log(np.mean(a)), logw0, 
                              logq, logsig[-1], 
                              tparams[0], tparams[1], 
                              tparams[2], tparams[3], a[1])

In [8]:
transit = utils.theano_transit(t, *tparams)
kernel = xo.gp.terms.SHOTerm(
            log_S0 = logs0[-1],
            log_w0 = logw0,
            log_Q=logq
        )
diag = np.exp(2*logsig[-1])*tt.ones(len(t))
#diag.eval()
gp = GP(kernel, t, diag, J=2)

In [ ]:
t = np.linspace(0, 10, 1000)
term = xo.gp.terms.SHOTerm(log_S0 = 0, log_w0 = 0, log_Q = -np.log(np.sqrt(2)))

q = np.array([1, 2, 3])
Q = q[:, None]*q[None, :]

diag = 0.001*tt.ones(len(t))

kernel = xo.gp.terms.KroneckerTerm(term, Q)

In [ ]:
gp1 = xo.gp.GP(x=t, kernel=kernel, diag=diag, J=6)
#gp2 = xo.gp.GP(x=t, kernel=term, diag=diag, J=6, Q=Q)

In [ ]:
n = np.random.randn(3*len(t), 1)
y1 = gp1.dot_l(n).eval()
#y2 = gp2.dot_l(n).eval()

In [ ]:
#print(np.max(y2 - y1))
#print(gp2.log_likelihood(y2).eval() - gp1.log_likelihood(y2).eval())

In [ ]:
with pm.Model() as model:
    logS0 = pm.Uniform("logS0", lower=-50.0, upper=50.0, testval=0)
    logw0 = pm.Uniform("logw0", lower=-50.0, upper=50.0, testval=0)
    logQ = pm.Uniform("logQ", lower=-50.0, upper=50.0, testval=-np.log(np.sqrt(2)))
    logdiag = pm.Uniform("logdiag", lower=-50.0, upper=50.0, testval=np.log(0.01))
    a = pm.Uniform("a", lower=1.0, upper=10.0, testval=2.0)
    b = pm.Uniform("b", lower=1.0, upper=10.0, testval=5.0)
    
    term = xo.gp.terms.SHOTerm(
            log_S0 = logS0,
            log_w0 = logw0,
            log_Q = logQ
        )
    
    q = tt.stack(1, a, b)
    Q = q[:, None]*q[None, :]
    diag = np.exp(logdiag)*tt.ones((3,len(t)))
    gp = xo.gp.GP(x=t, kernel=term, diag=diag, J=6, Q=Q)
    pm.Potential("loglike", gp.log_likelihood(y1.T))
    map_soln = xo.optimize(start=model.test_point, verbose=True)
map_soln

In [ ]:
with pm.Model() as model:
    logS0 = pm.Uniform("logS0", lower=-50.0, upper=50.0, testval=0)
    logw0 = pm.Uniform("logw0", lower=-50.0, upper=50.0, testval=0)
    logQ = pm.Uniform("logQ", lower=-50.0, upper=50.0, testval=-np.log(np.sqrt(2)))
    logdiag = pm.Uniform("logdiag", lower=-50.0, upper=50.0, testval=np.log(0.01))
    a = pm.Uniform("a", lower=1.0, upper=10.0, testval=2.0)
    b = pm.Uniform("b", lower=1.0, upper=10.0, testval=3.0)
    
    term = xo.gp.terms.SHOTerm(
            log_S0 = logS0,
            log_w0 = logw0,
            log_Q=logQ
        )
    
    q = tt.stack(1, a, b)
    Q = q[:, None]*q[None, :]
    diag = np.exp(logdiag)*tt.ones((3,len(t)))
    kern = xo.gp.terms.KroneckerTerm(term, Q)
    gp = xo.gp.GP(x=t, kernel=kern, diag=diag, J=6)
    pm.Potential("loglike", gp.log_likelihood(y1.T))
    map_soln = xo.optimize(start=model.test_point, verbose=True)
map_soln